In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab.csv')

In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:5].values
Y = data.iloc[:,5:].values

train_data,test_data,train_targets,test_targets=train_test_split(X,Y,test_size=0.2,random_state=42)
test_data,pred_data,test_targets,pred_targets=train_test_split(test_data,test_targets,test_size=0.5,random_state=42)

print(len(train_data))
print(test_data[0],test_targets[0])


In [ ]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

In [ ]:
!pip install keras-tuner

ANN modelling using boston method-Bayes opt


In [ ]:
#build network for bayesian testing
from keras import models
from keras import layers
from keras import Input
import keras_tuner
import keras


def build_model(hp):
  model=models.Sequential()
  model.add(Input(shape=(train_data.shape[1],)))
  for i in range(hp.Int('num_dense_layers', 1, 3)):
      acti='relu'
      acti=hp.Choice('activation',values=['relu','sigmoid','tanh'])
      model.add(layers.Dense(units=hp.Int('dense_' + str(i), min_value=20, max_value=120, step=10), activation=acti))
      model.add(layers.Dropout(hp.Choice('dropout_'+ str(i), values=[0.0, 0.2, 0.1])))
  model.add(layers.Dense(5)),#activation='relu') 
  
  hp_optimizer='adam'
  hp_optimizer=hp.Choice('Optimizer', values=['adam', 'Nadam','RMSprop'])

  
  #hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2])
 
  model.compile(optimizer=hp_optimizer,loss="mse",metrics="mae")
  return model
  

In [ ]:
my_callbacks = [
    keras.callbacks.EarlyStopping(patience=2),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    keras.callbacks.TensorBoard(log_dir='./logs'),
]


In [ ]:
tuner_mlp=keras_tuner.BayesianOptimization(build_model,objective="val_loss",max_trials=500,seed=42)

tuner_mlp.search(train_data, train_targets, epochs=500, batch_size=30, validation_data=(test_data, test_targets),callbacks=my_callbacks)



In [ ]:
best_mlp_hyperparameters = tuner_mlp.get_best_hyperparameters(1)[0]
print("Best Hyper-parameters")
best_mlp_hyperparameters.values

Best Hyper-parameters 9/4
{'num_dense_layers': 1,
 'activation': 'tanh',
 'dense_0': 120,
 'dropout_0': 0.1,
 'Optimizer': 'RMSprop',
 'dense_1': 120,
 'dropout_1': 0.0,
 'dense_2': 120,
 'dropout_2': 0.1}

Best Hyper-parameters 8/28
{'num_dense_layers': 1,
 'dense_0': 40,
 'dropout_0': 0.1,
 'Optimizer': 'adam',
 'dense_1': 110,
 'dropout_1': 0.1,
 'dense_2': 120,
 'dropout_2': 0.0}

Best Hyper-parameters 8/27
{'num_dense_layers': 3,
 'dense_0': 120,
 'dropout_0': 0.0,
 'Optimizer': 'adam',
 'learning_rate': 0.01,
 'dense_1': 120,
 'dropout_1': 0.1,
 'dense_2': 20,
 'dropout_2': 0.0}

In [ ]:
import tensorflow
def new_model():
    model_mlp=models.Sequential()
    model_mlp.add(Input(shape=(train_data.shape[1],)))
    model_mlp.add(layers.Dense(120,  activation='tanh'))
    model_mlp.add(layers.Dropout(0.1))
    model_mlp.add(layers.Dense(120,  activation='tanh'))
    model_mlp.add(layers.Dropout(0.0))
    model_mlp.add(layers.Dense(120,  activation='tanh'))
    model_mlp.add(layers.Dropout(0.1))
    model_mlp.add(layers.Dense(5))#,activation="softmax"))
    #tensorflow.keras.optimizers.Adam(learning_rate=0.01, name="adam")
    model_mlp.compile(optimizer="RMSprop", loss='mse',metrics='mae')
    return model_mlp


In [ ]:
import numpy as np

k=4
num_val_samples=len(train_data)//k
num_epochs=100
all_scores=[]

for i in range(k):
  print('processing fold#',i)
  val_data=train_data[i*num_val_samples:(i+1)*num_val_samples]
  val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
  partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)

  model=new_model()
  model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,verbose=0,batch_size=1)
  val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
  all_scores.append(val_mae)

In [ ]:
print(all_scores)
print(np.mean(all_scores))

In [ ]:
num_epochs = 2500
all_mae_histories = []
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)
  model = new_model()
  history = model.fit(partial_train_data, partial_train_targets,validation_data=(val_data, val_targets),epochs=num_epochs, batch_size=1, verbose=0)
  mae_history = history.history['val_mae']
  all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor)) 
    else:
      smoothed_points.append(point)
  return smoothed_points
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
model = new_model()
model.fit(train_data, train_targets,epochs=2200 , verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print(test_mae_score,test_mse_score)

In [ ]:
prediction=model.predict(pred_data)

In [ ]:
print(prediction)
print(pred_targets)
print(pred_data)
#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

LR_MAPE= MAPE(pred_targets,prediction)
print("MAPE: ",LR_MAPE)


In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/averagegc5in5outr4.h5') 
